#### Assignment submitted by : Adishree Garg & Vipul Suri of Batch: DS13-Aug-19

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
# import zipfile
# with zipfile.ZipFile(https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL, 'r') as zip_ref:
#     zip_ref.extractall(directory_to_extract_to)

In [4]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 30   #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(0, 30, 2) #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size                         # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),128,128,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    sized_img=imresize(image,(128,128))
                    
                    batch_data[folder,idx,:,:,0] = (sized_img[:,:,0])/255  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (sized_img[:,:,1])/255  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (sized_img[:,:,2])/255  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
#         rest_batch_size = len(folder_list)- (num_batches * batch_size)  # calculate the number of batches
        rest_batch_size = len(folder_list)%(batch_size)
        if (rest_batch_size!=0):
            batch_data = np.zeros((rest_batch_size,len(img_idx),128,128,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((rest_batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(rest_batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (rest_batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (rest_batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    sized_img=imresize(image,(128,128))
                    
                    batch_data[folder,idx,:,:,0] = (sized_img[:,:,0])/255  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (sized_img[:,:,1])/255  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (sized_img[:,:,2])/255  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (rest_batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30                                           # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


# Model 1
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [28]:
input_shape = (batch_size,15,128,128,3)[1:]

#write your model here
model = Sequential()
model.add(Conv3D(128, 
                 kernel_size=(5,5,5),
                 input_shape=input_shape,
                 padding='same', name="conv1"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(128, 
                 kernel_size=(5,5,5),
                 padding='same',name="conv2"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(128, 
                 kernel_size=(3,3,3), 
                 padding='same',name="conv3"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
#softmax layer
model.add(Dense(5, activation='softmax'))

In [29]:
optimiser = 'adam'                                                       #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 15, 128, 128, 128) 48128     
_________________________________________________________________
activation_10 (Activation)   (None, 15, 128, 128, 128) 0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 15, 128, 128, 128) 512       
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 7, 64, 64, 128)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 7, 64, 64, 128)    2048128   
_________________________________________________________________
activation_11 (Activation)   (None, 7, 64, 64, 128)    0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 7, 64, 64, 128)    512       
__________

### We will not train the model with above params because there are large no. of trainable params

## Model 2
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.optimizers import Adam

input_shape = (batch_size,15,128,128,3)[1:]

#write your model here
model = Sequential()
model.add(Conv3D(16, 
                 kernel_size=(3,3,3),
                 input_shape=input_shape,
                 padding='same', name="conv1"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, 
                 kernel_size=(2,2,2),
                 padding='same',name="conv2"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
#model.add(Dropout(0.25))

model.add(Conv3D(64, 
                 kernel_size=(2,2,2), 
                 padding='same',name="conv3"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
#softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = 'adam'                                                       #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 15, 128, 128, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 15, 128, 128, 16)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 128, 128, 16)  64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 64, 64, 16)     0         
_________________________________________________________________
conv2 (Conv3D)               (None, 7, 64, 64, 32)     4128      
_________________________________________________________________
activation_2 (Activation)    (None, 7, 64, 64, 32)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 64, 64, 32)     128       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.001)           # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 47s 2s/step - loss: 3.0727 - categorical_accuracy: 0.2366 - val_loss: 1.5920 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-07-1313_39_32.664967/model-00001-3.12927-0.24585-1.59198-0.23000.h5
Epoch 2/30
23/23 [==============================] - 40s 2s/step - loss: 1.6299 - categorical_accuracy: 0.1873 - val_loss: 1.6106 - val_categorical_accuracy: 0.1500

Epoch 00002: saving model to model_init_2020-07-1313_39_32.664967/model-00002-1.63067-0.19457-1.61061-0.15000.h5
Epoch 3/30
23/23 [==============================] - 41s 2s/step - loss: 1.6225 - categorical_accuracy: 0.1925 - val_loss: 1.6097 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-07-1313_39_32.664967/model-00003-1.62319-0.18703-1.60969-0.20000.h5
Epoch 4/30
23/23 [==============================] - 41s 2s/step - loss: 1.6101 - categorical_accuracy: 0.1858 - val_loss: 1.6106 - val_categorical_accuracy: 0.2100

Epoch 00004:

# Model 3

In [30]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30                                               # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [31]:
input_shape = (20,15,128,128,3)[1:]

model = Sequential()
model.add(Conv3D(32, (3, 3, 3), padding='same',activation = 'relu',
                 input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3, 3),activation = 'relu'))

model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (3, 3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())

model.add(Conv3D(64, (3, 3, 3), activation = 'relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (3, 3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.20))

model.add(Flatten())
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation = 'softmax'))

# model.add(Activation('softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [32]:
optimiser = 'sgd'                                                       #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 15, 128, 128, 32)  2624      
_________________________________________________________________
batch_normalization_18 (Batc (None, 15, 128, 128, 32)  128       
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 13, 126, 126, 32)  27680     
_________________________________________________________________
batch_normalization_19 (Batc (None, 13, 126, 126, 32)  128       
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 6, 63, 63, 32)     0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 6, 63, 63, 64)     55360     
_________________________________________________________________
batch_normalization_20 (Batc (None, 6, 63, 63, 64)     256       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [33]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [34]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.001)           # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [35]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [36]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30Source path =  Project_data/train ; batch size = 30

Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/23 [>.............................] - ETA: 2:47 - loss: 2.5928 - categorical_accuracy: 0.2333

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 79s 3s/step - loss: 1.7131 - categorical_accuracy: 0.2012 - val_loss: 1.6076 - val_categorical_accuracy: 0.1500

Epoch 00001: saving model to model_init_2020-07-1314_06_16.182384/model-00001-1.71717-0.19608-1.60759-0.15000.h5
Epoch 2/30
23/23 [==============================] - 72s 3s/step - loss: 1.6096 - categorical_accuracy: 0.2050 - val_loss: 1.6087 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init_2020-07-1314_06_16.182384/model-00002-1.60966-0.18703-1.60872-0.18000.h5
Epoch 3/30
23/23 [==============================] - 72s 3s/step - loss: 1.6072 - categorical_accuracy: 0.2253 - val_loss: 1.6097 - val_categorical_accuracy: 0.1500

Epoch 00003: saving model to model_init_2020-07-1314_06_16.182384/model-00003-1.60742-0.20814-1.60966-0.15000.h5
Epoch 4/30
23/23 [==============================] - 73s 3s/step - loss: 1.6075 - categorical_accuracy: 0.2143 - val_loss: 1.6094 - val_categorical_accuracy: 0.1800

Epoch 00004:

## Model 4
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [37]:
#write your model here
#Dividing data into X_train and y_train
# (x_train, y_train) = train_doc
# (x_test, y_test) = val_doc

input_shape = (batch_size,15,128,128,3)[1:]

model = Sequential()
model.add(Conv3D(32, (3, 3, 3), padding='same',activation = 'relu',
                 input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3, 3),activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(48, (3, 3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv3D(48, (3, 3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(96, (3, 3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())
# model.add(Conv3D(64, (3, 3, 3),padding='same', activation = 'relu'))
# model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.20))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
# model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation = 'softmax'))
# model.add(Activation('softmax'))
optimiser = 'sgd'                                                       #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_11 (Conv3D)           (None, 15, 128, 128, 32)  2624      
_________________________________________________________________
batch_normalization_23 (Batc (None, 15, 128, 128, 32)  128       
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 13, 126, 126, 32)  27680     
_________________________________________________________________
batch_normalization_24 (Batc (None, 13, 126, 126, 32)  128       
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 6, 63, 63, 32)     0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 6, 63, 63, 32)     0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 6, 63, 63, 48)     41520     
__________

In [38]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [39]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.001)           # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [40]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [41]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 79s 3s/step - loss: 7.8202 - categorical_accuracy: 0.2744 - val_loss: 2.7350 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2020-07-1314_06_16.182384/model-00001-8.02227-0.28507-2.73502-0.41000.h5
Epoch 2/30
23/23 [==============================] - 71s 3s/step - loss: 2.2329 - categorical_accuracy: 0.3368 - val_loss: 2.3408 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2020-07-1314_06_16.182384/model-00002-2.26245-0.34992-2.34076-0.22000.h5
Epoch 3/30
23/23 [==============================] - 71s 3s/step - loss: 1.4363 - categorical_accuracy: 0.4123 - val_loss: 1.5094 - val_categorical_accuracy: 0.2900

Epoch 00003: saving model to model_init_2020-07-1314_06_16.182384/model-00003-1.43960-0.42836-1.50937-0.29000.h5
Epoch 4/30
23/23 [==============================] - 71s 3s/step - loss: 1.1878 - categorical_accuracy: 0.5148 - val_loss: 4.4381 - val_categorical_accuracy: 0.2600

Epoch 00004:

# Model 5
using TimeDistributed while building a Conv2D + RNN model, the last layer is the softmax. Designing the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [42]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM
from keras.layers.convolutional import Conv3D,Conv2D, MaxPooling3D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers.recurrent import LSTM, GRU

In [44]:
#write your model here
nb_featuremap = [8,16,32,64]
nb_dense = [128,64,5]
nb_classes = 5
# Input
input_shape = (batch_size,15,128,128,3)[1:]

model = Sequential()

model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))


model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(nb_classes, activation='softmax')) # using Softmax as last layer

In [45]:
from keras.optimizers import Adam

optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 64, 64, 8)     224       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 15, 64, 64, 16)    1168      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 32, 32, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 32, 32, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 15, 16, 16, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 15, 16, 16, 64)    8256      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 15, 8, 8, 64)      0         
__________

In [46]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.001)           # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [47]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=5, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/23 [==>...........................] - ETA: 51s - loss: 1.5524 - categorical_accuracy: 0.3444 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 45s 2s/step - loss: 1.3554 - categorical_accuracy: 0.3900 - val_loss: 1.6954 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2020-07-1314_06_16.182384/model-00001-1.35643-0.39216-1.69541-0.37000.h5
Epoch 2/30
23/23 [==============================] - 38s 2s/step - loss: 1.0144 - categorical_accuracy: 0.5859 - val_loss: 2.5338 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2020-07-1314_06_16.182384/model-00002-1.00079-0.59578-2.53377-0.25000.h5
Epoch 3/30
23/23 [==============================] - 41s 2s/step - loss: 0.6869 - categorical_accuracy: 0.7146 - val_loss: 1.2884 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-07-1314_06_16.182384/model-00003-0.69393-0.71644-1.28838-0.55000.h5
Epoch 4/30
23/23 [==============================] - 41s 2s/step - loss: 0.5501 - categorical_accuracy: 0.7857 - val_loss: 1.1418 - val_categorical_accuracy: 0.6400

Epoch 00004: